In [1]:
### Loading Libraries

import os
import sys
import numpy                              as np
import pandas                             as pd

import matplotlib
from matplotlib                           import pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from tensorflow                           import keras

import romnet                             as rmnt

import cantera                            as ct

In [2]:
PathToDataFld  = WORKSPACE_PATH + '/ROMNet/Data/PSR_10Cases/3PC/'

In [3]:
class PSR(object):

    #===========================================================================
    def __init__(
        self,
        PathToDataFld
    ):
        PathToParams   = PathToDataFld + '/ROM/'

        
        self.mixture_file = 'gri30.yaml'
        self.fuel         = 'CH4:1.0'
        self.oxidizer     = 'O2:1.0, N2:0.0'
        self.T0_in        = 300.
        self.P0_in        = ct.one_atm
        self.eq_ratio_in  = 1.
        self.v            = 1.0

        
        
        gas                  = ct.Solution(self.mixture_file)
        self.n_species       = gas.n_species

        ### Create Inlet
        gas.TP               = self.T0_in, self.P0_in 
        gas.set_equivalence_ratio(self.eq_ratio_in, self.fuel, self.oxidizer, basis='mass')
        self.y_in            = gas.Y
        self.h_in            = np.dot(gas.X/gas.volume_mole, gas.partial_molar_enthalpies) / gas.density

        ### Create Combustor
        gas.equilibrate('HP')
        self.gas             = gas
        self.P               = gas.P
        self.h0              = np.dot(gas.X/gas.volume_mole, gas.partial_molar_enthalpies)/gas.density
        self.gas.HP          = self.h0, gas.P

        self.density_times_v = gas.density * self.v
        
        
        
        
        self.A         = pd.read_csv(PathToParams+'/A.csv', header=None).to_numpy()
        self.C         = pd.read_csv(PathToParams+'/C.csv', header=None).to_numpy().T
        self.D         = pd.read_csv(PathToParams+'/D.csv', header=None).to_numpy().T
        self.AT        = self.A.T
        self.n_pc      = self.A.shape[0]

        self.to_orig   = pd.read_csv(PathToParams+'/ToOrig_Mask.csv',   header=None).to_numpy(int)[:,0]
    
    
    #===========================================================================
    def f(self, t, y_orig, rest):
        n_points   = y_orig.shape[0]

        mass       = np.sum(y_orig[:,1:], axis=1)
        print(mass)

        dy_orig_dt = np.zeros_like(y_orig)
        for i_point in range(n_points):
            mass_i = mass[i_point]
            H_i    = y_orig[i_point,0]
            Y_i    = y_orig[i_point,1:]
            rest_i = 10.**rest[i_point,0]
            
            self.gas.HPY         = H_i/mass_i, self.P, Y_i/mass_i

            rho                  = self.gas.density
            wdot                 = self.gas.net_production_rates
            mdot                 = self.density_times_v / rest_i

            dy_orig_dt[i_point,0]  = (mass*self.h_in - H_i) / rest_i
            dy_orig_dt[i_point,1:] = wdot * self.gas.molecular_weights * self.v + (self.y_in - Y_i) * mdot

        return dy_orig_dt

    #===========================================================================
    
    
    #===========================================================================
    def f_pc(self, t, y_pc, rest):
        n_points               = y_pc.shape[0]

        y_masked               = np.matmul(y_pc, self.A) * self.D + self.C
        
        y_orig                 = np.zeros((n_points,self.n_species+1))
        y_orig[:,self.to_orig] = y_masked
        mass                   = np.sum(y_orig[:,1:], axis=1)

        dy_orig_dt             = np.zeros_like(y_orig)
        for i_point in range(n_points):
            mass_i = mass[i_point]
            H_i    = y_orig[i_point,0]
            Y_i    = y_orig[i_point,1:]
            rest_i = 10.**rest[i_point,0]

            self.gas.HPY           = H_i/mass_i, self.P, Y_i/mass_i
  
            rho                    = self.gas.density
            wdot                   = self.gas.net_production_rates
            mdot                   = self.density_times_v/rest_i
  
            dy_orig_dt[i_point,0]  = (mass_i*self.h_in - H_i) / rest_i
            dy_orig_dt[i_point,1:] = wdot * self.gas.molecular_weights * self.v + (self.y_in - Y_i) * mdot
   
        dy_masked_dt           = dy_orig_dt[:,self.to_orig]
        dy_pc_dt               = np.matmul(dy_masked_dt/self.D, self.AT)

        return dy_masked_dt

    #===========================================================================

In [4]:
psr = PSR(PathToDataFld)

In [5]:
y_pc = np.array([[2.701695105597581801e+02,-1.300384270965107732e-02,1.610054189797738592e-03]])
rest = np.array([[-5.0]])

dy_pc_dt = psr.f_pc(0., y_pc, rest)

In [6]:
y = np.array([[-7.981068568233224505e+04,5.827182460449132576e-04,2.003427880503316424e-04,2.493209523303316615e-03,1.069026308759098848e-02,6.333282060477100732e-03,2.828378328089143842e-02,6.108147167037001482e-06,2.910253397740119434e-07,9.171447794173098643e-13,1.103246270992535423e-13,6.872751591885412603e-14,7.146550692306891988e-15,5.812122470788831218e-14,5.807865636787473064e-15,1.742430508657903318e-02,1.981683879413244018e-02,6.597986952301323895e-08,5.560989260158240537e-10,3.230620077177212643e-14,8.318985469617597448e-16,7.833837161088560628e-16,3.567364201523519105e-18,2.923413390857086344e-17,2.602236329091941748e-21,3.390220294702945527e-22,4.375660056672233501e-26,7.753952612727701069e-28,1.134659916105115042e-15,2.626311961698442590e-16,1.844178372486215513e-18,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.138433842707861858e-38,1.846575768272914103e-40,1.369751160838939878e-20,7.704398002861705803e-22]])
rest = np.array([[-5.0]])

dy_dt = psr.f(0., y, rest)

[0.08583121]


In [7]:
dy_dt

array([[-6.33907912e+00, -5.00152913e+00, -1.71957091e+00,
        -2.13994880e+01,  6.77096619e+03, -5.43593707e+01,
        -2.42763169e+02, -5.24223892e-02, -2.50687552e-03,
        -7.87165031e-09, -9.46922582e-10, -1.94239549e-07,
        -6.69702821e-09, -3.18507260e-07,  1.72039890e+03,
        -1.49554911e+02, -1.70090329e+02, -5.66256309e-04,
        -4.73340691e-06, -2.77295203e-10, -7.14058038e-12,
        -6.72401859e-12, -3.06187464e-14, -2.50904512e-13,
        -2.23353898e-17, -2.90991999e-18, -3.75583956e-22,
        -6.65548274e-24, -9.73884737e-12, -2.25420213e-12,
        -1.58282676e-14,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.48488521e-3

In [8]:
dy_pc_dt

array([[-1.48546350e+06,  2.78429695e+00, -3.20127056e+00,
        -4.35671908e+01,  6.73768905e+03, -8.98811192e+01,
        -1.69757767e+02,  2.47212524e+00, -1.65862361e+00,
         2.61024039e-01, -8.28008171e-02,  1.39045612e+00,
        -4.94811050e-02,  3.28490230e+01,  1.64141833e+03,
        -5.33101528e+01, -2.23915687e+02,  2.16854050e+00,
         1.03422197e+01,  6.34359789e-01,  7.29458968e-03,
         9.24338350e-02,  6.06647595e-05,  1.66380599e-03,
         5.00304495e-04,  5.77288560e-03,  1.31232976e+00,
        -1.35955459e+00,  2.53615436e-01,  1.38101547e-02,
         3.88900267e-06, -5.86395575e-01, -4.29338888e-02,
         1.52133883e-04,  2.22548217e-05]])